In [1]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FYQBLYG8H to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "id": "63fc0c29-abe6-4170-b3bf-1cf85656b247",
    "isDefault": true,
    "name": "Visual Studio Enterprise",
    "state": "Enabled",
    "tenantId": "4e9843a2-758e-4a3b-8628-2fc4141091af",
    "user": {
      "name": "jmangia@me.com",
      "type": "user"
    }
  }
]


In [3]:
from azureml.core import Workspace, Experiment, Run, Datastore
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Found the config file in: /home/nbuser/library/config.json


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. cpucluster


In [20]:
experiment_name = "my-first-experiment"
run = list(Experiment(workspace = ws, name = experiment_name).get_runs())[0]

In [21]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
my-first-experiment,75aca6d8-b47a-4c4c-98be-154ddce7b53a,,Completed,Link to Azure Portal,Link to Documentation


In [5]:
#list all datastores registered in current workspace
datastores = ws.datastores
for name, ds in datastores.items():
    print(name, ds.datastore_type)

workspaceblobstore AzureBlob
workspacefilestore AzureFile
challenge1 AzureBlob


In [6]:
# Default datastore (Azure file storage)
#def_data_store = ws.get_default_datastore() 
def_data_store = Datastore(ws, "challenge1")
def_data_store

In [7]:
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData

input_data_reference = DataReference(
    datastore=def_data_store,
    data_reference_name="test_data",
    path_on_datastore="microsoft-malware-prediction/test.csv")

output_data = PipelineData(
    "output_data1",
    datastore=def_data_store,
    output_name="output_data1")

In [19]:
%%writefile batch_score.py

import argparse
import os

from azureml.core import Run
run = Run.get_context()
run.log("start batch score", 1)

print("In batch_score.py")
print("As a data scientist, this is where I use my scoring code.")

parser = argparse.ArgumentParser("scoring")

parser.add_argument("--input", type=str, help="input data")
parser.add_argument("--output", type=str, help="output data")
parser.add_argument("--param1", type=str, help="param 1")


args = parser.parse_args()

print("Input: %s" % args.input)
print("Output: %s" % args.output)
print("Param1: %s" % args.param1)

run.log("Input: %s" % args.input, 1)
run.log("Output: %s" % args.output, 1)
run.log("Param1: %s" % args.param1, 1)

run.log("start batch score", 2)
run.log("end batch score", 1)

run.complete()


Overwriting batch_score.py


In [20]:
from azureml.pipeline.steps.python_script_step import PythonScriptStep
from azureml.pipeline.core.graph import PipelineParameter

pipeline_param = PipelineParameter(
  name="pipeline_arg", 
  default_value=10)


firstStep = PythonScriptStep(
    script_name="batch_score.py",
    arguments=["--input", input_data_reference, "--output", output_data, "--param1", pipeline_param],
    inputs=[input_data_reference],
    outputs=[output_data],
    compute_target=compute_target,
    source_directory="."
)

In [21]:
from azureml.pipeline.core import Pipeline

pipeline1 = Pipeline(workspace=ws, steps=[firstStep])

In [22]:
# Submit the pipeline to be run
pipeline_run1 = Experiment(ws, "Test_pipeline_exp5").submit(pipeline1, show_output=True)
pipeline_run1.wait_for_completion()


Created step batch_score.py [4a8ad7f1][88da5417-58d8-48b9-95ca-db23b38d49be], (This step will run and generate new outputs)
Using data reference test_data for StepId [4f3d49fc][cfe7239c-0f2e-4945-943e-a824ce376816], (Consumers of this data are eligible to reuse prior runs.)
Submitted pipeline run: 815559b9-4706-4c9b-9138-4caa178eb37d
RunId: 815559b9-4706-4c9b-9138-4caa178eb37d
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/63fc0c29-abe6-4170-b3bf-1cf85656b247/resourceGroups/MLHack/providers/Microsoft.MachineLearningServices/workspaces/challenge1/experiments/Test_pipeline_exp5/runs/815559b9-4706-4c9b-9138-4caa178eb37d
Status: Running
....................................................................
Status: Finished
{'runId': '815559b9-4706-4c9b-9138-4caa178eb37d', 'status': 'Completed', 'startTimeUtc': '2019-04-22T19:14:18.831334Z', 'endTimeUtc': '2019-04-22T19:20:32.373366Z', 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': None, 'runType': 

'Finished'

In [ ]:
pipeline1.publish(name="Test Pipeline")

In [ ]:
response = requests.post(published_pipeline1.endpoint, 
    headers=aad_token, 
    json={"ExperimentName": "Test Pipeline",
        "ParameterAssignments": {"pipeline_arg": 20}})